# Enhanced Fraud Detection Models Tutorial 🚀

## 🎯 Learning Objectives
By the end of this tutorial, you will understand:

1. **Cost-Sensitive Learning** - Incorporating business costs into model training
2. **Advanced Models** - XGBoost and LightGBM with early stopping
3. **Comprehensive Evaluation** - Beyond accuracy to business impact
4. **Cross-Validation Strategies** - Robust model evaluation
5. **Model Selection** - Choosing models based on business objectives

## 📋 What This File Does
The `enhanced_fraud_models.py` file implements:

**💰 Cost-Sensitive Approach:**
- Different costs for false positives vs false negatives
- Investigation costs consideration
- ROI-based model selection

**🤖 Advanced Models:**
- XGBoost with early stopping
- LightGBM with class balancing
- Multiple baseline models for comparison

**📊 Business Metrics:**
- Total cost analysis
- Fraud prevention value
- Alert efficiency metrics
- Return on Investment (ROI)

**🔍 Comprehensive Evaluation:**
- Cross-validation with multiple metrics
- Cost-sensitive scoring
- Business impact analysis

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_auc_score, f1_score, precision_score, recall_score,
    make_scorer, fbeta_score, cohen_kappa_score
)
from sklearn.utils.class_weight import compute_class_weight

# Advanced models
import xgboost as xgb
import lightgbm as lgb

# For saving results
import joblib

# Visualization settings
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Enhanced Fraud Detection Models Tutorial")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 50)

## 1. Understanding Cost-Sensitive Learning

In fraud detection, not all errors are equal. Let's understand the business costs:

In [ ]:
# Define business costs
class BusinessCosts:
    """Define the business costs for fraud detection"""
    def __init__(self):
        # Cost of missing a fraud (False Negative)
        self.fraud_cost = 200  # Average fraud amount
        
        # Cost of false alarm (False Positive)
        self.false_positive_cost = 10  # Customer dissatisfaction, support time
        
        # Cost to investigate each alert
        self.investigation_cost = 30  # Manual review cost
        
    def calculate_total_cost(self, confusion_matrix):
        """Calculate total cost based on confusion matrix"""
        tn, fp, fn, tp = confusion_matrix.ravel()
        
        fraud_loss = fn * self.fraud_cost  # Missed frauds
        false_alarm_cost = fp * self.false_positive_cost  # False alarms
        investigation_cost = (tp + fp) * self.investigation_cost  # All alerts
        
        total_cost = fraud_loss + false_alarm_cost + investigation_cost
        
        return {
            'fraud_loss': fraud_loss,
            'false_alarm_cost': false_alarm_cost,
            'investigation_cost': investigation_cost,
            'total_cost': total_cost,
            'fraud_prevented': tp * self.fraud_cost,
            'net_benefit': tp * self.fraud_cost - total_cost
        }

# Initialize business costs
costs = BusinessCosts()

print("💰 Business Cost Structure:")
print(f"  • Missing a fraud (FN): ${costs.fraud_cost}")
print(f"  • False alarm (FP): ${costs.false_positive_cost}")
print(f"  • Investigation per alert: ${costs.investigation_cost}")
print("\n📊 This means:")
print("  • It's 20x more expensive to miss a fraud than to have a false alarm")
print("  • Every alert costs money to investigate")
print("  • We need to balance detection rate with operational costs")

In [ ]:
# Visualize cost implications
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Cost comparison
errors = ['False Negative\n(Miss Fraud)', 'False Positive\n(False Alarm)']
error_costs = [costs.fraud_cost, costs.false_positive_cost + costs.investigation_cost]
colors = ['#e74c3c', '#f39c12']

ax1.bar(errors, error_costs, color=colors)
ax1.set_title('Cost Comparison: FN vs FP', fontsize=14, fontweight='bold')
ax1.set_ylabel('Cost ($)')
for i, cost in enumerate(error_costs):
    ax1.text(i, cost + 5, f'${cost}', ha='center', fontweight='bold')

# ROI visualization
fraud_caught = np.arange(0, 101, 10)
false_alarms = fraud_caught * 5  # Assume 5 FP for each TP
fraud_prevented_value = fraud_caught * costs.fraud_cost
total_costs = false_alarms * (costs.false_positive_cost + costs.investigation_cost) + \
              fraud_caught * costs.investigation_cost
net_benefit = fraud_prevented_value - total_costs

ax2.plot(fraud_caught, fraud_prevented_value, label='Fraud Prevented Value', color='#27ae60', linewidth=2)
ax2.plot(fraud_caught, total_costs, label='Total Costs', color='#e74c3c', linewidth=2)
ax2.plot(fraud_caught, net_benefit, label='Net Benefit', color='#3498db', linewidth=3)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax2.set_xlabel('Number of Frauds Caught')
ax2.set_ylabel('Amount ($)')
ax2.set_title('Cost-Benefit Analysis', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Loading and Preparing Data

Let's load our credit card fraud dataset with a focus on cost-sensitive preparation:

In [ ]:
# Load the dataset
df = pd.read_csv('../creditcard.csv')

print("📊 Dataset Information:")
print(f"Total transactions: {len(df):,}")
print(f"Features: {df.shape[1]}")
print(f"Fraud rate: {df['Class'].mean()*100:.3f}%")
print(f"Average transaction amount: ${df['Amount'].mean():.2f}")
print(f"Average fraud amount: ${df[df['Class']==1]['Amount'].mean():.2f}")

# Calculate class weights for cost-sensitive learning
class_weights = compute_class_weight(
    'balanced', 
    classes=np.unique(df['Class']), 
    y=df['Class']
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

print(f"\n⚖️ Class Weights (for balanced learning):")
print(f"Normal transactions (0): {class_weights[0]:.2f}")
print(f"Fraud transactions (1): {class_weights[1]:.2f}")
print(f"Weight ratio: {class_weights[1]/class_weights[0]:.0f}:1")

In [ ]:
# Prepare features and split data
feature_columns = [col for col in df.columns if col not in ['Class']]
X = df[feature_columns]
y = df['Class']

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n📂 Data Split:")
print(f"Training set: {len(X_train):,} samples ({y_train.sum()} frauds)")
print(f"Test set: {len(X_test):,} samples ({y_test.sum()} frauds)")

# Create validation set for early stopping
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train, test_size=0.2, stratify=y_train, random_state=42
)

print(f"\n📊 Validation Split (for early stopping):")
print(f"Training: {len(X_train_split):,} samples")
print(f"Validation: {len(X_val_split):,} samples")

## 3. Creating Cost-Sensitive Scorer

Let's create a custom scorer that considers business costs:

In [ ]:
# Create cost-sensitive scorer
def cost_sensitive_score(y_true, y_pred):
    """
    Calculate cost-sensitive score based on business costs.
    Returns negative cost (lower is better for sklearn scorers).
    """
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Calculate total cost
    fraud_loss = fn * costs.fraud_cost  # Missed frauds
    false_alarm_cost = fp * costs.false_positive_cost  # False alarms
    investigation_cost = (tp + fp) * costs.investigation_cost  # All alerts
    
    total_cost = fraud_loss + false_alarm_cost + investigation_cost
    
    return -total_cost  # Negative because sklearn maximizes scores

# Create scorer for cross-validation
cost_scorer = make_scorer(cost_sensitive_score, greater_is_better=False)

# Define multiple scorers for comprehensive evaluation
scoring_metrics = {
    'f1': 'f1',
    'precision': 'precision',
    'recall': 'recall',
    'roc_auc': 'roc_auc',
    'cost': cost_scorer
}

print("✅ Cost-sensitive scorer created!")
print("📊 This scorer will help us find models that minimize business costs, not just maximize accuracy.")

## 4. Training Baseline Models with Cost-Sensitive Approach

Let's train several baseline models with class weights to handle imbalance:

In [ ]:
# Initialize results storage
models = {}
results = {}
cv_scores = {}

# Define baseline models
baseline_models = {
    'Logistic Regression': LogisticRegression(
        class_weight='balanced', 
        random_state=42, 
        max_iter=1000
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100, 
        class_weight='balanced', 
        random_state=42,
        n_jobs=-1
    ),
    'Decision Tree': DecisionTreeClassifier(
        class_weight='balanced', 
        random_state=42,
        max_depth=10
    ),
    'Naive Bayes': GaussianNB(),
    'Neural Network': MLPClassifier(
        hidden_layer_sizes=(100, 50),
        random_state=42,
        max_iter=500,
        early_stopping=True,
        validation_fraction=0.1
    )
}

print("🤖 Training Baseline Models with Cost-Sensitive Approach")
print("=" * 60)

In [ ]:
# Function to perform cross-validation
def perform_cross_validation(model, X, y, model_name, cv_folds=3):
    """Perform comprehensive cross-validation with multiple metrics."""
    print(f"\n🔄 Performing {cv_folds}-fold cross-validation for {model_name}")
    
    cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_results = {}
    
    for metric_name, scorer in scoring_metrics.items():
        scores = cross_val_score(model, X, y, cv=cv, scoring=scorer, n_jobs=-1)
        
        cv_results[f'{metric_name}_mean'] = scores.mean()
        cv_results[f'{metric_name}_std'] = scores.std()
        
        if metric_name == 'cost':
            print(f"  {metric_name}: ${-scores.mean():.0f} (+/- ${scores.std() * 2:.0f})")
        else:
            print(f"  {metric_name}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
    
    return cv_results

# Function to calculate business metrics
def calculate_business_metrics(y_true, y_pred, y_pred_proba=None):
    """Calculate comprehensive business and performance metrics."""
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    # Standard metrics
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    # Business cost metrics
    cost_details = costs.calculate_total_cost(cm)
    
    # ROI calculation
    roi = (cost_details['net_benefit'] / cost_details['total_cost']) * 100 if cost_details['total_cost'] > 0 else 0
    
    metrics = {
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'tp': tp,
        'fp': fp,
        'tn': tn,
        'fn': fn,
        'total_cost': cost_details['total_cost'],
        'fraud_prevented': cost_details['fraud_prevented'],
        'net_benefit': cost_details['net_benefit'],
        'roi': roi
    }
    
    if y_pred_proba is not None:
        metrics['roc_auc'] = roc_auc_score(y_true, y_pred_proba)
    
    return metrics

In [ ]:
# Train baseline models
for name, model in baseline_models.items():
    print(f"\n🚀 Training {name}")
    print("-" * 40)
    
    # Cross-validation
    cv_results = perform_cross_validation(model, X_train_scaled, y_train, name)
    cv_scores[name] = cv_results
    
    # Train final model
    model.fit(X_train_scaled, y_train)
    
    # Predictions
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
    
    # Calculate metrics
    metrics = calculate_business_metrics(y_test, y_pred, y_pred_proba)
    
    # Store results
    models[name] = model
    results[name] = metrics
    
    print(f"\n📊 Test Set Performance:")
    print(f"  F1-Score: {metrics['f1_score']:.4f}")
    print(f"  Total Cost: ${metrics['total_cost']:,.0f}")
    print(f"  ROI: {metrics['roi']:.1f}%")

## 5. Training Advanced Models with Early Stopping

Now let's train XGBoost and LightGBM with early stopping to prevent overfitting:

In [ ]:
# Calculate scale_pos_weight for imbalanced data
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

print("🚀 Training Advanced Models with Early Stopping")
print("=" * 60)
print(f"Scale positive weight: {scale_pos_weight:.2f}")

# XGBoost with early stopping
print("\n📊 Training XGBoost with early stopping...")

xgb_model = xgb.XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    early_stopping_rounds=20,
    eval_metric='logloss'
)

# Fit with evaluation set
xgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val_split, y_val_split)],
    verbose=False
)

print(f"✅ XGBoost stopped at iteration: {xgb_model.best_iteration}")

# Predictions
y_pred_xgb = xgb_model.predict(X_test_scaled)
y_proba_xgb = xgb_model.predict_proba(X_test_scaled)[:, 1]

# Metrics
xgb_metrics = calculate_business_metrics(y_test, y_pred_xgb, y_proba_xgb)
models['XGBoost'] = xgb_model
results['XGBoost'] = xgb_metrics

print(f"\n📊 XGBoost Test Performance:")
print(f"  F1-Score: {xgb_metrics['f1_score']:.4f}")
print(f"  ROC-AUC: {xgb_metrics['roc_auc']:.4f}")
print(f"  Total Cost: ${xgb_metrics['total_cost']:,.0f}")
print(f"  ROI: {xgb_metrics['roi']:.1f}%")

In [ ]:
# LightGBM with early stopping
print("\n📊 Training LightGBM with early stopping...")

lgb_model = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    class_weight='balanced',
    random_state=42,
    verbose=-1
)

# Fit with evaluation set and early stopping
lgb_model.fit(
    X_train_split, y_train_split,
    eval_set=[(X_val_split, y_val_split)],
    callbacks=[lgb.early_stopping(20), lgb.log_evaluation(0)]
)

print(f"✅ LightGBM stopped at iteration: {lgb_model.best_iteration_}")

# Predictions
y_pred_lgb = lgb_model.predict(X_test_scaled)
y_proba_lgb = lgb_model.predict_proba(X_test_scaled)[:, 1]

# Metrics
lgb_metrics = calculate_business_metrics(y_test, y_pred_lgb, y_proba_lgb)
models['LightGBM'] = lgb_model
results['LightGBM'] = lgb_metrics

print(f"\n📊 LightGBM Test Performance:")
print(f"  F1-Score: {lgb_metrics['f1_score']:.4f}")
print(f"  ROC-AUC: {lgb_metrics['roc_auc']:.4f}")
print(f"  Total Cost: ${lgb_metrics['total_cost']:,.0f}")
print(f"  ROI: {lgb_metrics['roi']:.1f}%")

## 6. Comprehensive Model Comparison

Let's compare all models from both ML performance and business perspectives:

In [ ]:
# Create comparison DataFrame
comparison_data = []

for model_name, metrics in results.items():
    row = {
        'Model': model_name,
        'Precision': metrics['precision'],
        'Recall': metrics['recall'],
        'F1-Score': metrics['f1_score'],
        'ROC-AUC': metrics.get('roc_auc', 0),
        'Total Cost': metrics['total_cost'],
        'Net Benefit': metrics['net_benefit'],
        'ROI (%)': metrics['roi'],
        'Alerts': metrics['tp'] + metrics['fp'],
        'Frauds Caught': metrics['tp'],
        'False Alarms': metrics['fp']
    }
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Total Cost')

print("📊 Comprehensive Model Comparison")
print("=" * 100)
print(comparison_df.round(4).to_string(index=False))

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. F1-Score comparison
ax1 = axes[0, 0]
comparison_df.sort_values('F1-Score', ascending=False).plot.bar(
    x='Model', y='F1-Score', ax=ax1, color='#3498db', legend=False
)
ax1.set_title('F1-Score by Model', fontsize=14, fontweight='bold')
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
ax1.set_ylim(0, 1)

# 2. Total Cost comparison
ax2 = axes[0, 1]
comparison_df.plot.bar(
    x='Model', y='Total Cost', ax=ax2, color='#e74c3c', legend=False
)
ax2.set_title('Total Cost by Model (Lower is Better)', fontsize=14, fontweight='bold')
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
ax2.set_ylabel('Total Cost ($)')

# 3. ROI comparison
ax3 = axes[0, 2]
comparison_df.sort_values('ROI (%)', ascending=False).plot.bar(
    x='Model', y='ROI (%)', ax=ax3, color='#27ae60', legend=False
)
ax3.set_title('Return on Investment by Model', fontsize=14, fontweight='bold')
ax3.set_xticklabels(ax3.get_xticklabels(), rotation=45, ha='right')
ax3.axhline(y=0, color='black', linestyle='--', alpha=0.5)

# 4. Precision vs Recall
ax4 = axes[1, 0]
ax4.scatter(comparison_df['Recall'], comparison_df['Precision'], s=200, alpha=0.6)
for idx, row in comparison_df.iterrows():
    ax4.annotate(row['Model'], (row['Recall'], row['Precision']), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)
ax4.set_xlabel('Recall')
ax4.set_ylabel('Precision')
ax4.set_title('Precision vs Recall Trade-off', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

# 5. Alert Volume Analysis
ax5 = axes[1, 1]
alert_data = comparison_df[['Model', 'Frauds Caught', 'False Alarms']]
alert_data.set_index('Model').plot.bar(stacked=True, ax=ax5, 
                                       color=['#27ae60', '#e74c3c'])
ax5.set_title('Alert Volume Analysis', fontsize=14, fontweight='bold')
ax5.set_ylabel('Number of Alerts')
ax5.set_xticklabels(ax5.get_xticklabels(), rotation=45, ha='right')
ax5.legend(['Frauds Caught', 'False Alarms'])

# 6. Cost Breakdown
ax6 = axes[1, 2]
best_model = comparison_df.iloc[0]['Model']
best_metrics = results[best_model]
cost_breakdown = [
    best_metrics['fn'] * costs.fraud_cost,
    best_metrics['fp'] * costs.false_positive_cost,
    (best_metrics['tp'] + best_metrics['fp']) * costs.investigation_cost
]
labels = ['Fraud Losses', 'False Alarm Cost', 'Investigation Cost']
ax6.pie(cost_breakdown, labels=labels, autopct='%1.1f%%', 
        colors=['#e74c3c', '#f39c12', '#3498db'])
ax6.set_title(f'Cost Breakdown - {best_model}', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Feature Importance Analysis

Let's analyze which features are most important for fraud detection:

In [ ]:
# Get feature importance from tree-based models
importance_models = ['Random Forest', 'XGBoost', 'LightGBM']
feature_importance_dict = {}

for model_name in importance_models:
    if model_name in models:
        model = models[model_name]
        if hasattr(model, 'feature_importances_'):
            feature_importance_dict[model_name] = model.feature_importances_

# Create feature importance DataFrame
feature_names = feature_columns
importance_df = pd.DataFrame(feature_importance_dict, index=feature_names)

# Calculate average importance
importance_df['Average'] = importance_df.mean(axis=1)
importance_df = importance_df.sort_values('Average', ascending=False)

# Plot top 15 features
plt.figure(figsize=(12, 8))
top_features = importance_df.head(15)

# Create horizontal bar plot
y_pos = np.arange(len(top_features))
plt.barh(y_pos, top_features['Average'], color='#3498db')
plt.yticks(y_pos, top_features.index)
plt.xlabel('Average Feature Importance')
plt.title('Top 15 Most Important Features for Fraud Detection', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='x')

# Add importance values
for i, v in enumerate(top_features['Average']):
    plt.text(v + 0.001, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()

print("🔝 Top 10 Most Important Features:")
print(top_features['Average'].head(10).to_string())

## 8. Business Recommendations

Based on our comprehensive analysis, let's provide actionable recommendations:

In [ ]:
# Find best model based on different criteria
best_f1_model = comparison_df.loc[comparison_df['F1-Score'].idxmax(), 'Model']
best_cost_model = comparison_df.loc[comparison_df['Total Cost'].idxmin(), 'Model']
best_roi_model = comparison_df.loc[comparison_df['ROI (%)'].idxmax(), 'Model']

print("💼 Business Recommendations")
print("=" * 60)

print(f"\n🏆 Best Models by Criteria:")
print(f"  • Best F1-Score: {best_f1_model} ({comparison_df[comparison_df['Model']==best_f1_model]['F1-Score'].values[0]:.4f})")
print(f"  • Lowest Cost: {best_cost_model} (${comparison_df[comparison_df['Model']==best_cost_model]['Total Cost'].values[0]:,.0f})")
print(f"  • Best ROI: {best_roi_model} ({comparison_df[comparison_df['Model']==best_roi_model]['ROI (%)'].values[0]:.1f}%)")

# Calculate potential savings
baseline_cost = comparison_df['Total Cost'].max()
best_cost = comparison_df['Total Cost'].min()
savings = baseline_cost - best_cost
savings_percent = (savings / baseline_cost) * 100

print(f"\n💰 Potential Cost Savings:")
print(f"  • Maximum savings: ${savings:,.0f} ({savings_percent:.1f}% reduction)")
print(f"  • Annual projection (assuming similar volume): ${savings * 365 / 2:,.0f}")

# Alert workload analysis
best_model_alerts = comparison_df[comparison_df['Model']==best_cost_model]['Alerts'].values[0]
total_transactions = len(y_test)
alert_rate = (best_model_alerts / total_transactions) * 100

print(f"\n📊 Operational Impact:")
print(f"  • Alert rate: {alert_rate:.2f}% of transactions")
print(f"  • Daily alerts (estimated): {best_model_alerts * 365 / 2:.0f}")
print(f"  • Precision: {comparison_df[comparison_df['Model']==best_cost_model]['Precision'].values[0]:.1%} of alerts are actual fraud")

print("\n🎯 Recommendations:")
print(f"  1. Deploy {best_cost_model} for production use")
print("  2. Set up monitoring for model drift and performance degradation")
print("  3. Consider A/B testing with current system")
print("  4. Implement feedback loop for continuous improvement")
print("  5. Review and adjust cost parameters quarterly")

## 9. Saving Models and Results

Let's save our trained models and results for future use:

In [ ]:
# Save all models and results
save_data = {
    'models': models,
    'scaler': scaler,
    'results': results,
    'comparison': comparison_df,
    'cost_config': {
        'fraud_cost': costs.fraud_cost,
        'false_positive_cost': costs.false_positive_cost,
        'investigation_cost': costs.investigation_cost
    },
    'feature_columns': feature_columns
}

# Save to file
joblib.dump(save_data, '../enhanced_fraud_models_tutorial.joblib')
print("✅ Models and results saved to 'enhanced_fraud_models_tutorial.joblib'")

# Save comparison report
comparison_df.to_csv('../model_comparison_report.csv', index=False)
print("✅ Comparison report saved to 'model_comparison_report.csv'")

## 10. Key Takeaways and Conclusions

### 🎯 What We Learned:

1. **Cost-Sensitive Learning is Crucial**:
   - Traditional accuracy metrics are misleading for imbalanced data
   - Business costs should drive model selection
   - ROI provides a holistic view of model value

2. **Advanced Models Excel**:
   - XGBoost and LightGBM often outperform traditional models
   - Early stopping prevents overfitting
   - Proper class balancing is essential

3. **Business Impact Matters**:
   - Lower F1-score might be acceptable if costs are lower
   - Alert volume affects operational capacity
   - False positive costs add up quickly

4. **Feature Importance Insights**:
   - PCA features (V-series) are highly predictive
   - Transaction amount plays a role
   - Time patterns exist but are subtle

### 💡 Best Practices:

1. **Always consider business costs** when selecting models
2. **Use multiple evaluation metrics** for comprehensive assessment
3. **Implement early stopping** for gradient boosting models
4. **Monitor feature importance** for model interpretability
5. **Plan for operational impact** of model deployment

### 🚀 Next Steps:

In the upcoming tutorials, you'll learn:
- **Model Calibration**: Adjusting prediction thresholds for optimal performance
- **Deep Learning**: Using autoencoders and neural networks
- **Real-time Systems**: Building production-ready APIs
- **Advanced Techniques**: Graph neural networks and ensemble methods

### 📝 Practice Exercises:

1. Try adjusting the business costs and see how model selection changes
2. Experiment with different XGBoost hyperparameters
3. Implement a custom threshold based on cost optimization
4. Create an ensemble of the top 3 models

## 🎉 Congratulations!

You've successfully implemented cost-sensitive fraud detection models! You now understand:
- How to incorporate business costs into ML workflows
- Advanced model training with early stopping
- Comprehensive evaluation beyond accuracy
- Making data-driven business recommendations

Ready to calibrate your models for production? Check out `advanced_model_calibration.ipynb`!